In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import bokeh
import os

In [ ]:
#La siguiente funcion lee las 3 carpetas de informacion y crea 3 dataframes, luego a cada uno les realiza la ventana movil
#para asi fusionarlos en un solo dataframe grande. Despues se escala una base solo con las variables numericas para finalmente 
#realizar los primeros modelos (regresion lineal y logistica).

In [2]:
def probando(file1,file2,file3,delta_movil):
    
    ######Cargar bases NORMAL, asignar etiqueta y ventana de tiempo.---------------------------------------------------------------------
    names_normal_files=os.listdir(file1)
    #Consolidando todas las bases con comportamiento normal en un DF
    consolidated_normal_df = pd.DataFrame()
    for name in names_normal_files:
        filename= f'df_{name[:-4]}'
        path = F'{file1}/{name}'
        filename = pd.read_csv(path)
        consolidated_normal_df = pd.concat([consolidated_normal_df,filename])
    
    #Asignando etiquetas
    consolidated_normal_df['Label']=0
    consolidated_normal_df['dataset']="Normal"
    
    #Ventana movil
    data_auxN=pd.DataFrame.copy(consolidated_normal_df)
    data_auxN['Time'] = data_auxN.index if data_auxN.index.is_monotonic_increasing else range(len(data_auxN))
    data_auxN.rename(columns={'Vm1:Measured voltage:1': 'V1', 'Vm1:Measured voltage:2':'V2', 
                                          'Vm1:Measured voltage:3':'V3','Divide18:1': 'IM1','Divide18:2':'IM2','Divide18:3': 'IM3',
                                          'Am1:Measured current': 'Corriente'}, inplace=True)
    for j in range(1,delta_movil+1):
        data_auxN2 = pd.DataFrame.copy(consolidated_normal_df)
        data_auxN2['Time']= (data_auxN2.index if data_auxN2.index.is_monotonic_increasing else range(len(data_auxN2)))
        data_auxN2['Time']=data_auxN2['Time']+j
        data_auxN2.rename(columns={'Vm1:Measured voltage:1': 'V1_'+str(j), 'Vm1:Measured voltage:2':'V2_'+str(j), 
                                            'Vm1:Measured voltage:3':'V3_'+str(j),'Divide18:1': 'IM1_'+str(j),'Divide18:2':'IM2_'+str(j),
                                            'Divide18:3': 'IM3_'+str(j),'Am1:Measured current': 'Corriente_'+str(j), 'Label': 'Label_'+str(j),
                                            'dataset': 'dataset_'+str(j)}, inplace=True)
        data_auxN3 = data_auxN.merge(data_auxN2, on='Time', how='inner')
        data_auxN = pd.DataFrame.copy(data_auxN3)
    #return data_auxN

    
    ######Cargar bases ATAQUE, asignar etiqueta y ventana de tiempo.---------------------------------------------------------------------
    names_attacked_files=os.listdir(file2) 
    #Consolidando todas las bases con comportamiento atacado en un DF
    consolidated_attacked_df = pd.DataFrame()
    for name in names_attacked_files:
        filename= f'df_{name[:-4]}'
        path = f'Attacked_Operation/{name}'
        filename = pd.read_csv(path)
        consolidated_attacked_df = pd.concat([consolidated_attacked_df,filename])
    
    #Funcion para etiquetar si la medición esta bajo ataque o no (sirve para este item como para las bases desbalanceadas)
    def label_data(time):
        if time <= 0.3:
            return 0
        else:
            return 1
    consolidated_attacked_df['Label']=consolidated_attacked_df['Time'].apply(label_data)

    #Por temas de testeo, no necesitamos datos despues de los 0,5 segundos por lo que deben ser removidos.
    consolidated_attacked_df.reset_index(drop=True,inplace=True)
    boolean_mask = consolidated_attacked_df['Time']>0.5
    attacked_index = consolidated_attacked_df[boolean_mask].index
    consolidated_attacked_df.drop(attacked_index, axis=0,inplace=True)
    
    #Ahora etiqueto que es una base con ataque
    consolidated_attacked_df['dataset'] = "Attacked"

    #Ventana móvil
    data_auxA=pd.DataFrame.copy(consolidated_attacked_df)
    data_auxA['Time'] = data_auxA.index if data_auxA.index.is_monotonic_increasing else range(len(data_auxA))
    data_auxA.rename(columns={'Vm1:Measured voltage:1': 'V1', 'Vm1:Measured voltage:2':'V2', 
                                          'Vm1:Measured voltage:3':'V3','Divide18:1': 'IM1','Divide18:2':'IM2','Divide18:3': 'IM3',
                                          'Am1:Measured current': 'Corriente'}, inplace=True)
    for j in range(1,delta_movil+1):
        data_auxA2 = pd.DataFrame.copy(consolidated_attacked_df)
        data_auxA2['Time']= (data_auxA2.index if data_auxA2.index.is_monotonic_increasing else range(len(data_auxA2)))
        data_auxA2['Time']=data_auxA2['Time']+j
        data_auxA2.rename(columns={'Vm1:Measured voltage:1': 'V1_'+str(j), 'Vm1:Measured voltage:2':'V2_'+str(j), 
                                            'Vm1:Measured voltage:3':'V3_'+str(j),'Divide18:1': 'IM1_'+str(j),'Divide18:2':'IM2_'+str(j),
                                            'Divide18:3': 'IM3_'+str(j),'Am1:Measured current': 'Corriente_'+str(j),'Label': 'Label_'+str(j),
                                            'dataset': 'dataset_'+str(j)}, inplace=True)
        data_auxA3 = data_auxA.merge(data_auxA2, on='Time', how='inner')
        data_auxA = pd.DataFrame.copy(data_auxA3)
    #return data_auxA
    
    ######Cargar bases DESBALANCEADAS ATACADAS, asignar etiqueta y ventana de tiempo.---------------------------------------------------------------------
    names_imbalanced_files=os.listdir(file3)
    #Consolidando todas las bases desbalanceadas con comportamiento atacado en un DF
    consolidated_imbalanced_df = pd.DataFrame()
    for name in names_imbalanced_files:
        filename= f'df_{name[:-4]}'
        path = f'Imbalanced_Attacked/{name}'
        filename = pd.read_csv(path)
        consolidated_imbalanced_df = pd.concat([consolidated_imbalanced_df,filename])
    
    #Aplico etiqueta si esta bajo ataque o no
    consolidated_imbalanced_df['Label']=consolidated_imbalanced_df['Time'].apply(label_data)
    
    #Por temas de testeo, no necesitamos datos despues de los 0,5 segundos por lo que deben ser removidos.
    consolidated_imbalanced_df.reset_index(drop=True,inplace=True)
    boolean_im_mask = consolidated_imbalanced_df['Time']>0.5
    imbalanced_index = consolidated_imbalanced_df[boolean_im_mask].index
    consolidated_imbalanced_df.drop(imbalanced_index,axis=0,inplace=True)
    
    #Ahora etiqueto que es una base desbalanceada con ataque
    consolidated_imbalanced_df['dataset'] = "Imbalanced"
    
    #Ventana móvil
    data_auxIA=pd.DataFrame.copy(consolidated_imbalanced_df)
    data_auxIA['Time'] = data_auxIA.index if data_auxIA.index.is_monotonic_increasing else range(len(data_auxIA))
    data_auxIA.rename(columns={'Vm1:Measured voltage:1': 'V1', 'Vm1:Measured voltage:2':'V2', 
                                          'Vm1:Measured voltage:3':'V3','Divide18:1': 'IM1','Divide18:2':'IM2','Divide18:3': 'IM3',
                                          'Am1:Measured current': 'Corriente'}, inplace=True)
    for j in range(1,delta_movil+1):
        data_auxIA2 = pd.DataFrame.copy(consolidated_imbalanced_df)
        data_auxIA2['Time']= (data_auxIA2.index if data_auxIA2.index.is_monotonic_increasing else range(len(data_auxIA2)))
        data_auxIA2['Time']=data_auxIA2['Time']+j
        data_auxIA2.rename(columns={'Vm1:Measured voltage:1': 'V1_'+str(j), 'Vm1:Measured voltage:2':'V2_'+str(j), 
                                            'Vm1:Measured voltage:3':'V3_'+str(j),'Divide18:1': 'IM1_'+str(j),'Divide18:2':'IM2_'+str(j),
                                            'Divide18:3': 'IM3_'+str(j),'Am1:Measured current': 'Corriente_'+str(j),'Label': 'Label_'+str(j),
                                            'dataset': 'dataset_'+str(j)}, inplace=True)
        data_auxIA3 = data_auxIA.merge(data_auxIA2, on='Time', how='inner')
        data_auxIA = pd.DataFrame.copy(data_auxIA3)
    #return data_auxIA    

    ###### Consolidando todos los dataframes, con ventana de tiempo, en uno--------------------------------------------------------
    consolidated_df = pd.concat([data_auxN, data_auxA]) #bases normales con atacadas
    consolidated_df = pd.concat([consolidated_df, data_auxIA]) #lo anterior + desbalanceado
    #print(consolidated_df.Label.value_counts())
    
    #####Reseteando los indices
    consolidated_df.reset_index(drop=True,inplace=True)
    #print(consolidated_df)
    
    
    #ESTO ES LA PRUEBA TRATANDO DE ESCALAR, EN LA SIGUIENTE LINEA DE CODIGO VEREMOS QUE PASA CON EL MODELO SIN ESCALAR.
    
    
    ##### Preporcesamiento de la data
    #Importando para escalar
    from sklearn.preprocessing import StandardScaler
    scaler=StandardScaler()
    
    #Solo se necesita escalar las variables numericas, asi que se realiza eso
    numerical_consolidated_df=consolidated_df.drop(['Time','Label','dataset'],axis=1)
    
    numerical_consolidated_df2=pd.DataFrame.copy(numerical_consolidated_df)
    
    for j in range(1,delta_movil+1):
        numerical_consolidated_df2=numerical_consolidated_df2.drop(['dataset_'+str(j),'Label_'+str(j)],axis=1)
    #return numerical_consolidated_df2
    
    #Escalar solo los valores numericos y colocar de vuelta a un df lo que era string
    scaled_consolidated_df=scaler.fit_transform(numerical_consolidated_df2)
    scaled_consolidated_df = pd.DataFrame(scaled_consolidated_df,columns=numerical_consolidated_df2.columns)
    #print(scaled_consolidated_df)
    
    #Volver a concatenar las columnas de string
    consolidated_df2=pd.concat([scaled_consolidated_df,consolidated_df[['Time','Label','dataset']]],axis=1)
    #print(consolidated_df2)
    for j in range(1,delta_movil+1):
        consolidated_df2=pd.concat([consolidated_df2,consolidated_df[['dataset_'+str(j),'Label_'+str(j)]]],axis=1)
    #return consolidated_df2
    
    
    ######### Testing First models -------------------------------------------------------------------------------------------
    
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import classification_report
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import accuracy_score
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import mean_squared_error
    from sklearn.model_selection import cross_val_score
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import SVC
    from sklearn.model_selection import train_test_split

    
    from statsmodels.stats.weightstats import ttest_ind
    res_ttest = ttest_ind(
                    x1 = consolidated_df2.V3[consolidated_df2.Label == 0],
                    x2 = consolidated_df2.V3[consolidated_df2.Label == 1],
                    alternative='two-sided'
                )
    print("Para entrenamiento de los datos con regresión logística se tienen los siguientes resultados:")
    print(f"t={res_ttest[0]}, p-value={res_ttest[1]}") #viendo el valor t y p
    
    ##### Train, test, split the data -----------------------------------------------------------------------------------------
    X=scaled_consolidated_df 
    y=consolidated_df2['Label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=100)
    
    ##Regresion LOGISTICA:
    
    #Creacion y entrenamiento del modelo logístico
    reg_log = LogisticRegression() #cargo el modelo de reg log
    reg_log.fit(X_train, y_train) #Entreno el clasificador
    
    print("Valor del intercepto:", reg_log.intercept_)
    #print("Valores de los Coeficientes:", list(zip(X.columns, reg_log.coef_.flatten(), )))
    print(f"Accuracy de entrenamiento para Reg Log: {100*(reg_log.score(X, y))}%")
    
    #Prediccion
    y_pred = reg_log.predict(X_train) #guardo las predicciones con el modelo entrenado
    
    print("")
    print("Para testeo de los datos con regresión logística se tienen los siguientes resultados:")
    #Accuracy de testeo
    #X_test = sm.add_constant(X_test, prepend=True) #no se en realidad si esto se debe agregar o no
    pred_test_log = reg_log.predict(X_test)
    clasificacion = np.where(pred_test_log<0.5, 0, 1)
    accuracy_test_reglog = accuracy_score(
            y_true    = y_test,
            y_pred    = clasificacion,
            normalize = True
           )
    print(f"El accuracy de testeo en Reg Log es: {100*accuracy_test_reglog}%")
    print("")
    
    
    ## SUPPORT VECTOR MACHINE (SVM)
    
    #Creacion y entrenamiento del modelo SVM
    svm_model=SVC(random_state=100)
    svm_model.fit(X_train,y_train)
    
    #Prediccion y accuracy en testeo:
    pred_test_svm=svm_model.predict(X_test)
    accuracy_test_svm = accuracy_score(
            y_true    = y_test,
            y_pred    = pred_test_svm,
            normalize = True
           )
    print(f"El accuracy de testeo en SVM es: {100*accuracy_test_svm}%")
    
    
    ##RANDOM FOREST:
    
    #Creacion y entrenamiento del modelo Random Forest
    random_forest = RandomForestClassifier(
            n_estimators = 100,
            max_depth    = None,
            max_features = 'auto',
            oob_score    = False,
            n_jobs       = -1,
            random_state = 100
         )

    random_forest.fit(X_train, y_train)
    
    
    #Prediccion y accuracy en testeo:
    pred_test_rf=random_forest.predict(X_test)
    accuracy_test_rf = accuracy_score(
            y_true    = y_test,
            y_pred    = pred_test_rf,
            normalize = True
           )
    print(f"El accuracy de testeo en Random Forest es: {100*accuracy_test_rf}%")
    
    
    ## K MEANS:
    KNN_model = KNeighborsClassifier(n_neighbors=5)
    KNN_model.fit(X_train, y_train)
    KNN_test_pred = KNN_model.predict(X_test)
    
    #Accuracy en testeo
    accuracy_test = accuracy_score(
            y_true    = y_test,
            y_pred    = KNN_test_pred,
            normalize = True
           )
    print(f"El accuracy de testeo en KNN es: {100*accuracy_test}%")
    

     #Matriz de confusion Regresion Logistica
    matrix1=confusion_matrix(y_test,pred_test_log)
    ax1= plt.subplot(2,2,1)
    sns.heatmap(matrix1, annot=True, fmt='g', ax=ax1);
    # labels, title and ticks
    ax1.set_xlabel('Predicted labels');ax1.set_ylabel('True labels'); 
    ax1.set_title('Confusion Matrix Reg Log'); 
    ax1.xaxis.set_ticklabels(['attack', 'normal']); ax1.yaxis.set_ticklabels(['normal', 'attack']);
    
    #Matriz de confusion SVM
    matrix2=confusion_matrix(y_test,pred_test_svm)
    ax2= plt.subplot(2,2,2)
    sns.heatmap(matrix2, annot=True, fmt='g', ax=ax2);
    # labels, title and ticks
    ax2.set_xlabel('Predicted labels');ax2.set_ylabel('True labels'); 
    ax2.set_title('Confusion Matrix SVM'); 
    ax2.xaxis.set_ticklabels(['attack', 'normal']); ax2.yaxis.set_ticklabels(['normal', 'attack']);
    
    #Matriz de confusion Random Forest
    matrix3=confusion_matrix(y_test,pred_test_rf)
    ax3= plt.subplot(2,2,3)
    sns.heatmap(matrix3, annot=True, fmt='g', ax=ax3);
    # labels, title and ticks
    ax3.set_xlabel('Predicted labels');ax3.set_ylabel('True labels'); 
    ax3.set_title('Confusion Matrix RF'); 
    ax3.xaxis.set_ticklabels(['attack', 'normal']); ax3.yaxis.set_ticklabels(['normal', 'attack']);
    
    #Matriz de confusion KNN
    matrix4=confusion_matrix(y_test,KNN_test_pred)
    ax4= plt.subplot(2,2,4)
    sns.heatmap(matrix4, annot=True, fmt='g', ax=ax4);
    # labels, title and ticks
    ax4.set_xlabel('Predicted labels');ax4.set_ylabel('True labels'); 
    ax4.set_title('Confusion Matrix KNN'); 
    ax4.xaxis.set_ticklabels(['attack', 'normal']); ax4.yaxis.set_ticklabels(['normal', 'attack']);
    
    
    
    #print(accuracy_score(KNN_prediction, y_test))
    #Confusion Matrix and Classification Report give more details about performance
    #print(classification_report(KNN_prediction, y_test))
    
#     from tabulate import tabulate
#     print("Ok, probando si sale una tabla")
    
#     d=[[accuracy_test_reglog,accuracy_test_svm]]

    
    
    
probando("Normal_Operation","Attacked_Operation","Imbalanced_Attacked",2)

FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: 'Normal_Operation'